# Convert MLD output to MDM output and visualize it

## MDM output format

In [3]:
import numpy as np

mdm_data = np.load('/home/ctq566/motion-diffusion-model/save/humanml_trans_enc_512/samples_humanml_trans_enc_512_000200000_seed10_The_man_spun_in_place_three_times/results.npy', allow_pickle=True).item()
# print all keys and value type of mdm_data, if it has shape attribute, print its shape in the same line, otherwise print its value
for key, value in mdm_data.items():
    if hasattr(value, 'shape'):
        print(key, type(value), value.shape)
    else:
        print(key, type(value), value)
        

motion <class 'numpy.ndarray'> (3, 22, 3, 120)
text <class 'list'> ['The man spun in place three times.', 'The man spun in place three times.', 'The man spun in place three times.']
lengths <class 'numpy.ndarray'> (3,)
num_samples <class 'int'> 1
num_repetitions <class 'int'> 3


## MLD output format

In [4]:
mld_data = np.load('/home/ctq566/motion-diffusion-model/playground/Example_50_batch0_0.npy', allow_pickle=True)
mld_data.shape


(50, 22, 3)

In [7]:
mld_data_dict = {'motion': mld_data.transpose((1, 2, 0)).reshape((1, 22, 3, 50)), 'text': 'No text', 'lengths': np.asarray([50]), 'num_samples': 1, 'num_repetitions': 1}
np.save('results.npy', mld_data_dict)


## The conversion and the stick animation generation

In [1]:
import os
import sys
sys.path.append('/home/ctq566/motion-diffusion-model/')
import numpy as np
import data_loaders.humanml.utils.paramUtil as paramUtil
from data_loaders.humanml.utils.plot_script import plot_3d_motion

def mld_to_mdm(mld_data_path):
    assert isinstance(mld_data_path, str)
    mld_data = np.load(mld_data_path, allow_pickle=True)
    lengths = mld_data.shape[0]
    mld_data_dict = {'motion': mld_data.transpose((1, 2, 0)).reshape((1, 22, 3, lengths)), 'text': 'No text', 'lengths': np.asarray([lengths]), 'num_samples': 1, 'num_repetitions': 1}
    # save the result to the same directory as results.npy
    np.save(f'{os.path.dirname(mld_data_path)}/results.npy', mld_data_dict)

    animation_save_path = f'{os.path.dirname(mld_data_path)}/sample00_rep00.mp4'
    skeleton = paramUtil.t2m_kinematic_chain
    motion = mld_data
    dataset = 'humanml'
    title = 'No title'
    fps = 20
    plot_3d_motion(animation_save_path, skeleton, motion, title, dataset, fps=fps)

    
mld_to_mdm('/home/ctq566/motion-diffusion-model/playground/Example_50_batch0_0.npy')


## Batch conversion

In [22]:
import glob
import tqdm


def batch_convert_mld_to_mdm(mld_data_dir):
    assert isinstance(mld_data_dir, str)
    assert os.path.isdir(mld_data_dir)
    print(f'Converting mld data in {mld_data_dir} to mdm data')
    for mld_data_path in tqdm.tqdm(glob.glob(f'{mld_data_dir}/*.npy')):
        # create a new directory for each mld data
        new_dir = f'{mld_data_dir}/{os.path.basename(mld_data_path).split(".")[0]}'
        os.mkdir(new_dir)
        # move the mld data to the new directory
        os.rename(mld_data_path,
                  f'{new_dir}/{os.path.basename(mld_data_path)}')
        # change the mld_data_path to the new directory
        mld_data_path = f'{new_dir}/{os.path.basename(mld_data_path)}'
        # convert the mld data to mdm data
        mld_to_mdm(mld_data_path)
    print('Done')


batch_convert_mld_to_mdm(
    '/home/ctq566/motion-diffusion-model/playground/samples_2023-02-22-21-58-06')


Converting mld data in /home/ctq566/motion-diffusion-model/playground/samples_2023-02-22-21-58-06 to mdm data


100%|██████████| 467/467 [35:36<00:00,  4.57s/it]

Done
